In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Autoinflammatory_Disorders"
cohort = "GSE80060"

# Input paths
in_trait_dir = "../../input/GEO/Autoinflammatory_Disorders"
in_cohort_dir = "../../input/GEO/Autoinflammatory_Disorders/GSE80060"

# Output paths
out_data_file = "../../output/preprocess/Autoinflammatory_Disorders/GSE80060.csv"
out_gene_data_file = "../../output/preprocess/Autoinflammatory_Disorders/gene_data/GSE80060.csv"
out_clinical_data_file = "../../output/preprocess/Autoinflammatory_Disorders/clinical_data/GSE80060.csv"
json_path = "../../output/preprocess/Autoinflammatory_Disorders/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression data of whole blood of systemic juvenile idiopathic arthritis (SJIA) patients treated with canakinumab or placebo and age matched healthy controls"
!Series_summary	"Canakinumab is a human anti-interleukin-1 beta (IL-1 beta) monoclonal antibody neutralizing IL-1 beta. Systemic juvenile idiopathic arthritis (SJIA) is a rare, multigenic, autoinflammatory disease of unknown etiology characterized by chronic arthritis; intermittent high-spiking fever, rash, and elevated levels of acute-phase reactants. Blood samples of SJIA patients were obtained from two phase 3 clinical trials conducted by the members of the Pediatric Rheumatology International Trials Organization (PRINTO) and the Pediatric Rheumatology Collaborative Study Group (PRCSG) (Clinicaltrials.gov: NCT00886769 and NCT00889863). For patients, baseline and day 3 samples were analyzed for either placebo or canakinumab (Ilaris) treatment."
!Series_summary	"Clinical response was a

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Libraries needed
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Determine if gene expression data is available
# Based on the background information, this is a gene expression dataset from whole blood
is_gene_available = True

# 2. Variable availability and data type conversion
# 2.1 Identify keys for trait, age, and gender
# For trait, we can use disease status at index 1
trait_row = 1  # Disease status (SJIA vs Healthy)

# Age and gender are not explicitly provided in the sample characteristics
age_row = None  # Age not available
gender_row = None  # Gender not available

# 2.2 Data type conversion functions
def convert_trait(value):
    """Convert disease status to binary (1 for SJIA, 0 for Healthy)"""
    if value is None:
        return None
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'SJIA' in value:
        return 1  # SJIA
    elif 'Healthy' in value:
        return 0  # Healthy control
    else:
        return None  # Unknown or not applicable

# Since age and gender are not available, we still define placeholder functions
def convert_age(value):
    return None

def convert_gender(value):
    return None

# 3. Save metadata about dataset usability
# Determine trait availability based on trait_row
is_trait_available = trait_row is not None

# Save initial filtering results
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (only if trait_row is not None)
if trait_row is not None:
    # Create a sample characteristics dictionary from the output of previous step
    sample_characteristics_dict = {
        0: ['tissue: Whole blood'], 
        1: ['disease status: SJIA', 'disease status: Healthy'], 
        2: ['subject id: SJIA_2_2513', 'subject id: SJIA_2_313', 'subject id: SJIA_2_413', 'subject id: SJIA_2_712', 
            'subject id: SJIA_2_812', 'subject id: SJIA_2_912', 'subject id: SJIA_2_1013', 'subject id: SJIA_2_1112', 
            'subject id: SJIA_2_2912', 'subject id: SJIA_2_3012', 'subject id: SJIA_2_1413', 'subject id: SJIA_2_1411', 
            'subject id: SJIA_2_168', 'subject id: SJIA_2_167', 'subject id: SJIA_2_1713', 'subject id: SJIA_2_1811', 
            'subject id: SJIA_2_185', 'subject id: SJIA_2_1912', 'subject id: SJIA_2_2213', 'subject id: SJIA_2_2313', 
            'subject id: SJIA_2_2312', 'subject id: SJIA_2_113', 'subject id: SJIA_2_2613', 'subject id: SJIA_2_212', 
            'subject id: SJIA_2_310', 'subject id: SJIA_2_36', 'subject id: SJIA_2_512', 'subject id: SJIA_2_511', 
            'subject id: SJIA_2_613', 'subject id: SJIA_2_612'], 
        3: ['visit: Day1_BL', 'visit: Day3'], 
        4: ['treatment: Canakinumab', 'treatment: Placebo', 'treatment: none'], 
        5: ['acr response at day 15: 100', 'acr response at day 15: NA', 'acr response at day 15: 30', 
            'acr response at day 15: 70', 'acr response at day 15: 90', 'acr response at day 15: 0', 
            'acr response at day 15: 50']
    }
    
    # From the output of previous step, determine max length needed for the DataFrame
    max_len = max(len(values) for values in sample_characteristics_dict.values())
    
    # Create a DataFrame with appropriate dimensions
    clinical_data = pd.DataFrame(index=range(max_len))
    
    # Add each feature as a column, padding shorter lists with None
    for key, values in sample_characteristics_dict.items():
        # Extend shorter lists with None values
        padded_values = values + [None] * (max_len - len(values))
        clinical_data[f'feature_{key}'] = padded_values
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    preview_data = preview_df(selected_clinical_df)
    print("Preview of selected clinical data:")
    print(preview_data)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical data:
{'feature_0': [nan], 'feature_1': [0.0], 'feature_2': [1.0], 'feature_3': [nan], 'feature_4': [nan], 'feature_5': [nan]}
Clinical data saved to ../../output/preprocess/Autoinflammatory_Disorders/clinical_data/GSE80060.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers appear to be probe set IDs from an Affymetrix microarray
# These are not human gene symbols and would need to be mapped to proper gene symbols

# Looking at identifiers like '1007_s_at', '1053_at', etc., these follow the pattern
# of Affymetrix probe IDs which need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe gene identifiers and annotation to identify relevant columns
# The gene expression data has identifiers like '1007_s_at' in the index (using 'ID' as key)
# The gene annotation dataframe has a column 'ID' with the same format identifiers
# The 'Gene Symbol' column contains the gene symbols we need to map to

# 2. Get gene mapping dataframe
# Extract relevant columns for mapping from the gene annotation dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# Preview the mapping dataframe
print("Gene mapping preview (first 5 rows):")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the number of genes after mapping
print(f"Number of genes after mapping: {len(gene_data)}")

# Show a sample of the mapped gene expression data
print("Sample of gene expression data after mapping (first 5 genes):")
print(gene_data.head())


Gene mapping preview (first 5 rows):
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A


Number of genes after mapping: 21278
Sample of gene expression data after mapping (first 5 genes):
          GSM2111993  GSM2111994  GSM2111995  GSM2111996  GSM2111997  \
Gene                                                                   
A1BG       20.531274   14.741187   11.079894   10.599425   11.433046   
A1BG-AS1   88.194790   47.904730   81.249540   43.318430   47.088590   
A1CF        7.832409    7.952191   10.122947    7.881265    8.623113   
A2M        75.558405   56.413898   77.866086   53.556840   61.194481   
A2M-AS1   172.515520  345.047360  205.631310  344.107630  429.584250   

          GSM2111998  GSM2111999  GSM2112000  GSM2112001  GSM2112002  ...  \
Gene                                                                  ...   
A1BG       15.974882   16.569076   14.513591   10.338256   13.122920  ...   
A1BG-AS1   60.742230   71.671910   65.129780   69.265340   56.305330  ...   
A1CF       10.524899    8.914323    8.883382    7.754067    9.206250  ...   
A2M        

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# First, normalize gene symbols using the function from the library
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Original gene count: {len(gene_data)}")
print(f"Normalized gene count: {len(normalized_gene_data)}")

# Create directory for the gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the normalized gene data to a CSV file
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. We need to first load or recreate the selected_clinical_df
try:
    # Try to load the previously saved clinical data
    selected_clinical_df = pd.read_csv(out_clinical_data_file)
except:
    # If loading fails, recreate the clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)

    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)

# Check if linking was successful
if len(linked_data) == 0 or trait not in linked_data.columns:
    print("Linking clinical and genetic data failed - no valid rows or trait column missing")
    # Set is_usable to False and save cohort info
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=True,  # Consider it biased if linking fails
        df=pd.DataFrame({trait: [], 'Gender': []}), 
        note="Data linking failed - unable to process gene expression data appropriately."
    )
    print("The dataset was determined to be not usable for analysis.")
else:
    # 3. Handle missing values in the linked data
    linked_data = handle_missing_values(linked_data, trait)
    
    print(f"Linked data shape after handling missing values: {linked_data.shape}")
    
    # 4. Determine whether the trait and demographic features are severely biased
    trait_type = 'binary' if len(linked_data[trait].unique()) <= 2 else 'continuous'
    if trait_type == 'binary':
        if len(linked_data[trait].value_counts()) >= 2:
            is_trait_biased = judge_binary_variable_biased(linked_data, trait)
        else:
            print(f"Trait '{trait}' has only one unique value, considering it biased.")
            is_trait_biased = True
    else:
        is_trait_biased = judge_continuous_variable_biased(linked_data, trait)
    
    # Remove biased demographic features
    unbiased_linked_data = linked_data.copy()
    if 'Age' in unbiased_linked_data.columns:
        age_biased = judge_continuous_variable_biased(unbiased_linked_data, 'Age')
        if age_biased:
            print(f"The distribution of the feature \'Age\' in this dataset is severely biased.")
            unbiased_linked_data = unbiased_linked_data.drop(columns=['Age'])
    
    if 'Gender' in unbiased_linked_data.columns:
        if len(unbiased_linked_data['Gender'].value_counts()) >= 2:
            gender_biased = judge_binary_variable_biased(unbiased_linked_data, 'Gender')
            if gender_biased:
                print(f"The distribution of the feature \'Gender\' in this dataset is severely biased.")
                unbiased_linked_data = unbiased_linked_data.drop(columns=['Gender'])
        else:
            print(f"Gender has only one unique value, considering it biased and removing.")
            unbiased_linked_data = unbiased_linked_data.drop(columns=['Gender'])
    
    # 5. Conduct quality check and save the cohort information.
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=is_trait_biased, 
        df=unbiased_linked_data, 
        note="Dataset contains gene expression data from whole blood of systemic juvenile idiopathic arthritis (SJIA) patients treated with canakinumab or placebo and healthy controls."
    )
    
    # 6. If the linked data is usable, save it as a CSV file.
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("The dataset was determined to be not usable for analysis due to bias in the trait distribution.")

Original gene count: 21278
Normalized gene count: 19845


Normalized gene data saved to ../../output/preprocess/Autoinflammatory_Disorders/gene_data/GSE80060.csv
Linking clinical and genetic data failed - no valid rows or trait column missing
Abnormality detected in the cohort: GSE80060. Preprocessing failed.
The dataset was determined to be not usable for analysis.
